In [7]:
'''
#Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

# path = get_file(
#     'nietzsche.txt',
#     origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
# with io.open(path, encoding='utf-8') as f:
#     text = f.read().lower()
with open('./Lermontov all (raw text).txt', encoding='utf-8') as f:
    text = f.read()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(170, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=170,
          epochs=60,
          callbacks=[print_callback])

corpus length: 829812
total chars: 178
nb sequences: 276591
Vectorization...
Build model...
Epoch 1/60
276591/276591 [==============================] - 150s 541us/step - loss: 2.3099

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " глядится,
Но если свежий ветерок
Волну "
 глядится,
Но если свежий ветерок
Волну старой стороной столоной
С безумной старой столоной столодной
И сторовал страстный столоной
Не вольно старой своих столокой
И столоной своей столоной
И столодной старой столоной
Не старов стольно стоит старой,
Когда старой страсть старой столоной
И столодной страстный столоной
С полость столодной стороком
С отвой столон старой столоной
И столодной столоком столоной
Стоил старой страстной столоной

----- diversity: 0.5
----- Generating with seed: " глядится,
Но если свежий ветерок
Волну "
 глядится,
Но если свежий ветерок
Волну спенье зарыл сонь,
И тот стару не зарыл вороста
Заршился страстной грудь столодной,
Когда сына своих горокой,
Не с

И одну лишь свободню оно!
Я крик быступный дорогии;
Когда в ояз дерев, что нита,
О голосе мольбам,
Буду за ними боже, голоно он небы младрары
Терсились струраму,
Почти ее звезды,
Но пста прорасных сон,
Оставать дикина с для зрел.
348

И несцу, еся краска младожа.
Так я любви — вохоти!..
484

КИПГВДаны красналась рамурать.
И им ожидно, гарина своей
С нам гроблива глушенный
Но помеш я в чинних жизних;
Так для бы 
----- diversity: 1.2
----- Generating with seed: "могу в небесах я парить
И одну лишь своб"
могу в небесах я парить
И одну лишь свобьднепом,
Не платеноджвый мо! не испыл нечалеки;
О земных герочья в душе воозну;
Я поточно землю бредной,
*

выступтено эток отец,
Но тушь тихо мрачно билской
Два это сих всёры неналасем невзгляска,
Душа влеящие дровятсьва
Новородки, кто задумной ЗназР
суды гранисвой волер свету
Не простор птицы на новом,
Молча могила, и рапуле. Дна
4х8и будет он!.,
6зы телся — ь сам ,
Над переблыск.пРелской вад не
Epoch 5/60
276591/276591 [========================

В полным стали страстей страны
На страстей последних лет.
И в полны молча столой,
И последний в ней страсти,
И страсти в полна не видел,
И стал был свой пострадал скалы,
Как постой последний раз,
На столой страсти страстей свет,
И
----- diversity: 0.5
----- Generating with seed: "й,
Он именинник — будут гости...
От удив"
й,
Он именинник — будут гости...
От удивает помни свет
Насказали в молните свой страны
страсть в душе своей
На нем ветр не потом столой страстей
На нем женных столой, от стен,
И потом пестре страстей меж любви.
Ты славы, полны страна,
Да без туман, страстей заказал,
И было твоей любви, со мной
В тень деницы вечерней столокой,
В сердце судьбою волны,
Не взглянул будет мой последний лет;
В синей прочью рукой серебой
Солнце ля перемедит из
----- diversity: 1.0
----- Generating with seed: "й,
Он именинник — будут гости...
От удив"
й,
Он именинник — будут гости...
От удивлю нежный мечты
Сердца... и сме, равые :
Света какой пожуй высыу!
Нощней большой светит лодей!
Зачем не 

Мой нет свои вторишпо пы—?
Ни далей лотные х там кусчан.
Страдал я пощаснный достиней...
Сародов врекаженья, люби
Туский выше его разни измераетс,
Туман! семью мгновенья с сколкала
С заброшеней неверовие!
Девиц, двою лежета
Уки? монатлись вошку от—
Хдады, иль был вы
Между сия моем молиток
В тихка тоской уж радскизя,
И смирый взгляд он счастье этот взор;
Что быстрочной, рокопот в комных,
Одо крыльни в че
Epoch 12/60
276591/276591 [==============================] - 127s 459us/step - loss: 1.6465

----- Generating text after Epoch: 11
----- diversity: 0.2
----- Generating with seed: "
Иль женщин уважать возможно,
Когда мне "

Иль женщин уважать возможно,
Когда мне старик под него,
И в страхе своей последних стране
И полны старик открыты
И волноственный свой свой волно
Свободно пред ним не подоболья!
Не взор не стал на стрелами,
Не отрадил по славеская всегда
И с тобой старик возвратила,
И в толпе устала на свете
И в тени бледные странных надеждом,
Не полетел последних слез,
И полна не взо

Как наш без толппет он сырой,
Что признал он к нему сокрылись,
Как будто полны был он, в нем,
Как полны безумный раздался
Седой слупая в душе на давно,
Как на свой кистит надел в нем и вершины.
Валив
----- diversity: 1.0
----- Generating with seed: " бумажки.
И... вы легко поймете радость "
 бумажки.
И... вы легко поймете радость чада
над безвеслил.
4

Порой, чем не узГаша друзьба, но жжать,
Как иль твоиц противы радуям!
Оныне чашей ссед субилих,
Оснугтов кибтренивший день...

Я в комучит ты возьму. С них, где храмит,
Расспалье, им прощаде, на кинуть
Меня отвозузвых, ни выое.
И, обольца меле умчить,
Но чуждых сад никто сзот, тут было проолыбик;
Покорница, как сильно моления,
Как р закам
От право ледяных пели,
Ему, что были
----- diversity: 1.2
----- Generating with seed: " бумажки.
И... вы легко поймете радость "
 бумажки.
И... вы легко поймете радость падь
Прежченья обманно верник,
Хочут обнять ровы ж молне нас с другами...
123

«Неисчаваник позданья!..
Кимлетысь пред богамней.— Чтож,

...

Плывет прошло обманутой,
Как полный скал и толпой, но голосовой
И красный высокой и волненья
И мир не всем не помнить,
И всё среди меня и в нем странной,
На белорерных полей свон,
И славы не создана и глупей страны,
И в нем полно и твой странной дум

Из предчасти не вольность моим,
И всё был уныло он не слышал
И волновалась с собой,
И то видела не свет,
И старикой образ свободной!..

450
эЬю серни стрел
----- diversity: 1.0
----- Generating with seed: " он,
Как пногда черкес чрез Терек
Плывет"
 он,
Как пногда черкес чрез Терек
Плывет их недвижима божиеной
Всегоами моих подруга,
Брезлет... коно — будто не мог...
Помню изгнанник безумной и ступени,
Вздыхает рожден, и битва был,
Алжет у, а друг мелет, ветро жизнью долином
Не я в тучки настается
Кровим п слах?
Надежно уздно ль?
Расчедка горой строгогот.
17

Проду по нах, и я изменит он,
Ходил потослуво, тучи множило сног низесток.
И как жегранна, потом.
Люблю я поднял пред любов
----- diversity: 1.2
----- Generating with seed: " он,
Как п

Едва кенец и блещут,
С камница руко холодный подал:
«Пуклю чупотьм песнь — своей
Epoch 49/60
276591/276591 [==============================] - 127s 459us/step - loss: 1.5184

----- Generating text after Epoch: 48
----- diversity: 0.2
----- Generating with seed: "но плащом,
Чтобы из них никто сказать не"
но плащом,
Чтобы из них никто сказать не верил,
И все помня, но сказал он не зная,
И всё свою страданье пришла,
Пришла не мог его страдал,
Где не поднял в темных стран,
И слезы в поле поднялась,
Поверить не станет в темном,
И слезы пред ним от вечер не страшись,
И с стенах не создан на славине страдает
И полны в ней пред ним от всех очей,
И в темном тех порвания в коварах,
Когда он вечно не просил,
Всё пришел последних следы мое страшно
----- diversity: 0.5
----- Generating with seed: "но плащом,
Чтобы из них никто сказать не"
но плащом,
Чтобы из них никто сказать не была
И стала бы скорей, с тобою
В шас рад, резвит полночит печальный,
Воспоминанье в душе моей:
Но скажи сердце страдал...

Врагу заснувшему он вдруг свой лег;
И бучту — и бледна; наследишь.

На мять дявящий свой свой
Черкаю моя не посвятой,
Скользой, ни бросил вот о роками,
Как лих усбал Дни, грядущий,
Не отключий в бировальным.
Я орец их неиз них душой,
То, что я те могучий разгоневоры?
Было спасених стологом,
С тобой только нельзя с дикой.
Я раз мнодадлевней грозной,
По море ветра незнак было,
Ушпалы? — Измаил толькое голучий
По меденым
----- diversity: 1.2
----- Generating with seed: "едолго рассуждает;
Врагу заснувшему он в"
едолго рассуждает;
Врагу заснувшему он всё: любя друж
Мых кудри были сведкат.
А вдруг уклежает листь. Гухою мой тем
Дит: «Ходи страшку с Имитехиле —
И не завтрастный лавтю ни куршим,
Казалось, он проры мне. Чтобы оправесь пожир,
Уроки — занарискавшись близку-веческой,
Во все его мне слезник,
мгновен, меч станьстину длуде!
Во думу оставлено.
Еще волнуевсты на ним демоно любазат! —
И приведет житель Измаили
Кричями ранная его судьба!
В Ма
Epoch 53/60
276591/276591 [==================

С ним простой странные стологой,
Когда он в первый раз на своих лежали,
Как бы он вздохнул в нем страшный,
В одной старину темной стологой,
И в темном было в нем полетить,
И все приветство вечность стало.

423


Я вечно 
----- diversity: 0.5
----- Generating with seed: "уде тел
Они без сожаленья повлекли,
И по"
уде тел
Они без сожаленья повлекли,
И поклонело истокан башни туманным
На степпе при душе не верил.
Смотрит вышине не создал,
Вдруг взор неведомый взор —
И взгляд кровь в жизни железны.

310

1261

ПТОВОЙI В Капи ты одной ветер своей полна,
С ней было которая молодость,
Когда сердце потому и всё при плетели
Покрытый взор в мирным в тени,
Стоит в темной прибегает он,
Хоть верений умел с бурюбною,
Под ним и сердце разлукв до бегут над н
----- diversity: 1.0
----- Generating with seed: "уде тел
Они без сожаленья повлекли,
И по"
уде тел
Они без сожаленья повлекли,
И послушался языкомчат:
Стоит и бранная правд...
Ах!.. Но звонкий пыль вот любви
Его опастить онв тамары злоды?м, за 

В колена ей склонялася порою
Знатьу, — вай навяста,
Он пылки, часй!..
Я здесь, с холодных дних быт быть
Из светит: «Есть, мелькаяй, к какой.
Но опладало бромар врагом
От бул, тропалкит ланитый,
Он сахши злобною головой:
Показалась, ль стогля.!, у минута!..
Чесет там одиже сын, отчизненной,
И я небесовск они!
Пмы нем любовь жил, сыгой,
Друзей; властью странною тихо
Привершных часных векем,
Хадгрбы! 
Копскую яев, где я той, 
Epoch 60/60
276591/276591 [==============================] - 127s 459us/step - loss: 1.5135

----- Generating text after Epoch: 59
----- diversity: 0.2
----- Generating with seed: "торая освобождает его от очей моих;
неум"
торая освобождает его от очей моих;
неумчался бы в стране предо мной
И слезы привык пробудясь,
И всё в нем привык уж покровом
И всё приведел привык,
И взор твоих он сказал,
И взор на свете страстный страх
Надежат на свете страстью,
И взор на свете их подобный страх
Надежды и всегда под ним привык.

410

ВРОНА

Не отчизна сказать в нем страстей,
Ка

In [9]:
model.save('lermonet_3.h5')